In [1]:
library(tidyverse)
library(stringr)
library(ggplot2)
library(data.table)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




In [2]:
spectrum_generator <- function(count=100,coverage=2/3,offset=0,reverse=FALSE) {
    tau = 2*pi
    color = vector('list',3)
    coverage = ((-1)^reverse)*coverage
    offset = ((-1)^reverse)*offset
    for (iter in 1:3) {
        values = c()
        theta = (offset + iter - 1) * tau/3
        for (x in seq(0,tau*coverage,tau*coverage/(count-1))[1:count]) {
            values = c(values, cos(x + theta)/2 + 0.5)
        }
        color[[iter]] = values
    }
    return(rgb(color[[1]],color[[2]],color[[3]]))
}

In [15]:
statsfile_path = "../PD_ukb_step2_BT_ltfh.regenie"

In [16]:
gwas_raw <- fread(statsfile_path, sep=" ")
names(gwas_raw)[names(gwas_raw) == "CHROM"] <- "CHR"
names(gwas_raw)[names(gwas_raw) == "GENPOS"] <- "BP"

gwas_data <- gwas_raw

In [17]:
don <- gwas_data %>% 
  
  # Compute chromosome size
  group_by(CHR) %>% 
  summarise(chr_len=max(BP)) %>% 
  
  # Calculate cumulative position of each chromosome
  mutate(tot=cumsum(as.numeric(chr_len))-chr_len) %>%
  select(-chr_len) %>%
  
  # Add this info to the initial dataset
  left_join(gwas_data, ., by=c("CHR"="CHR")) %>%
  
  # Add a cumulative position of each SNP
  arrange(CHR, BP) %>%
  mutate(BPcum=BP+tot)

axisdf <- don %>% group_by(CHR) %>% summarize(center=( max(BPcum) + min(BPcum) ) / 2 )

spectrum <- spectrum_generator(
    count = 22,
    coverage = 5/6,
    offset = -0.25,
    reverse = TRUE
)

manplot <- ggplot(don, aes(x=BPcum, y=LOG10P)) +
    
    # Show all points
    geom_point( aes(color=as.factor(CHR)), alpha=1, size=0.2) +
    scale_color_manual(values = spectrum) +
    
    # custom X axis
    scale_x_continuous( label = axisdf$CHR, breaks=axisdf$center ) +
    scale_y_continuous(expand = c(0, 0) ) +

    # Custom theme
    theme_bw() +
    theme(
        legend.position = "none",
        panel.border = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank()
    ) +

    # Titles
    ggtitle("Manhattan Plot of Raw LT-FH PD GWAS Results") +
    xlab("Chromosome") +
    ylab("Log(p)")

In [ ]:
ggsave('../PD_manplot.png', plot=manplot, width=24, height=8, units='cm', dpi=600)

In [7]:
statsfile_path = "../PD_ukb_step2_BT_ltfh.regenie.corrected"

In [8]:
gwas_raw <- fread(statsfile_path, sep=" ")
names(gwas_raw)[names(gwas_raw) == "CHROM"] <- "CHR"
names(gwas_raw)[names(gwas_raw) == "GENPOS"] <- "BP"

gwas_data <- gwas_raw

In [9]:
don <- gwas_data %>% 
  
  # Compute chromosome size
  group_by(CHR) %>% 
  summarise(chr_len=max(BP)) %>% 
  
  # Calculate cumulative position of each chromosome
  mutate(tot=cumsum(as.numeric(chr_len))-chr_len) %>%
  select(-chr_len) %>%
  
  # Add this info to the initial dataset
  left_join(gwas_data, ., by=c("CHR"="CHR")) %>%
  
  # Add a cumulative position of each SNP
  arrange(CHR, BP) %>%
  mutate(BPcum=BP+tot)

axisdf <- don %>% group_by(CHR) %>% summarize(center=( max(BPcum) + min(BPcum) ) / 2 )

spectrum <- spectrum_generator(
    count = 22,
    coverage = 5/6,
    offset = -0.25,
    reverse = TRUE
)

manplot <- ggplot(don, aes(x=BPcum, y=LOG10P)) +
    
    # Show all points
    geom_point( aes(color=as.factor(CHR)), alpha=1, size=0.2) +
    scale_color_manual(values = spectrum) +
    
    # custom X axis
    scale_x_continuous( label = axisdf$CHR, breaks=axisdf$center ) +
    scale_y_continuous(expand = c(0, 0) ) +

    # Custom theme
    theme_bw() +
    theme(
        legend.position = "none",
        panel.border = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank()
    ) +

    # Titles
    ggtitle("Manhattan Plot of Corrected LT-FH PD GWAS Results") +
    xlab("Chromosome") +
    ylab("Log(p)")

In [10]:
ggsave('../PD_corrected_manplot.png', plot=manplot, width=24, height=8, units='cm', dpi=600)

In [11]:
statsfile_path = "../PD_ukb_step2_BT_ltfh.regenie.corrected.reduced"

In [12]:
gwas_raw <- fread(statsfile_path, sep=" ")
names(gwas_raw)[names(gwas_raw) == "CHROM"] <- "CHR"
names(gwas_raw)[names(gwas_raw) == "GENPOS"] <- "BP"

In [13]:
don <- gwas_data %>% 
  
  # Compute chromosome size
  group_by(CHR) %>% 
  summarise(chr_len=max(BP)) %>% 
  
  # Calculate cumulative position of each chromosome
  mutate(tot=cumsum(as.numeric(chr_len))-chr_len) %>%
  select(-chr_len) %>%
  
  # Add this info to the initial dataset
  left_join(gwas_data, ., by=c("CHR"="CHR")) %>%
  
  # Add a cumulative position of each SNP
  arrange(CHR, BP) %>%
  mutate(BPcum=BP+tot)

axisdf <- don %>% group_by(CHR) %>% summarize(center=( max(BPcum) + min(BPcum) ) / 2 )

spectrum <- spectrum_generator(
    count = 22,
    coverage = 5/6,
    offset = -0.25,
    reverse = TRUE
)

manplot <- ggplot(don, aes(x=BPcum, y=LOG10P)) +
    
    # Show all points
    geom_point( aes(color=as.factor(CHR)), alpha=1, size=0.2) +
    scale_color_manual(values = spectrum) +
    
    # custom X axis
    scale_x_continuous( label = axisdf$CHR, breaks=axisdf$center ) +
    scale_y_continuous(expand = c(0, 0) ) +

    # Custom theme
    theme_bw() +
    theme(
        legend.position = "none",
        panel.border = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank()
    ) +

    # Titles
    ggtitle("Manhattan Plot of Corrected & Reduced LT-FH PD GWAS Results") +
    xlab("Chromosome") +
    ylab("Log(p)")

In [14]:
ggsave('../PD_corrected_reduced_manplot.png', plot=manplot, width=24, height=8, units='cm', dpi=600)